<a href="https://colab.research.google.com/github/jkordonis/TropicalFactorization/blob/main/MovieLens1million.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import cupy as np
#import numpy as np


In [22]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/shenweichen/DeepMatch/master/examples/preprocess.py -O preprocess.py
! unzip -o ml-1m.zip

import pandas as pd


NotImplementedError: ignored

In [ ]:
data_path = "./"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

In [ ]:
ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
indices_pd = ratings[['user_id', 'movie_id']].values
IND=np.array(indices_pd).astype(int)
IND_x,IND_y = IND[:,0],IND[:,1]


In [ ]:
n=6040
p=3952
B=np.zeros((n,p))+1

B[IND_x-1,IND_y-1]=-1

In [ ]:
n*p*0.9

In [ ]:
pairs=[(i,j) for j in range(p) for i in range(n)]
pairs=np.array(pairs)
np.random.shuffle(pairs)

N=19096064 #n*p*0.8
N_=21483072
N__=n*p



In [ ]:
IND_x_train = pairs[:N,0]
IND_y_train = pairs[:N,1]


IND_x_val  = pairs[N:N_,0]
IND_y_val  = pairs[N:N_,1]

IND_x_tst  = pairs[N_:N__,0]
IND_y_tst  = pairs[N_:N__,1]

B_sparse_ob_or =  B[IND_x_train,IND_y_train]
B_sparse_ob_or_val =  B[IND_x_val,IND_y_val]
B_sparse_ob_or_tst = B[IND_x_tst,IND_y_tst]


In [23]:
# Initialize C,D matrix estimates
m=35
r=100
C_0,D_Tr_0=np.random.rand(n,r).astype('float16'),np.random.rand(p,r).astype('float16')

In [24]:
C=np.copy(C_0)
D_Tr=np.copy(D_Tr_0)
Val_err=1000000

In [25]:
C=np.copy(C_0)
D_Tr=np.copy(D_Tr_0)
#C=np.copy(C_op)
#D_Tr = np.copy(D_op)

import time
T=time.time()
cnt=0
Err =np.zeros(100000)
Err_val =np.zeros(100000)
Err_tst =np.zeros(100000)

for _ in range(100):
  for _ in range(1):
    INDICES=np.array(range(N))
    np.random.shuffle(INDICES)
    BatchSize=1000
    i=0
    minibatches=[]
    while i*BatchSize<=N:
      minibatches.append(INDICES[i*BatchSize:BatchSize*(i+1)])
      i+=1

    #minibatches = [INDICES[i*BatchSize:BatchSize*(i+1)] for i in range(np.ceil(N/BatchSize).astype(int))]
    for AA in minibatches:
      IND_x_ob=IND_x_train[AA]
      IND_y_ob=IND_y_train[AA]
      B_sparse_ob=B_sparse_ob_or[AA]

      Maximizers = np.argmax(C[IND_x_ob]+D_Tr[IND_y_ob],-1)
      B_sparse_ob_est = C[IND_x_ob,Maximizers]+D_Tr[IND_y_ob,Maximizers]
      Differences = B_sparse_ob_est -B_sparse_ob
    #  for i in range(IND_x_ob.shape[0]):
      C_to_change = C[IND_x_ob,Maximizers]
      D_to_change = D_Tr[IND_y_ob,Maximizers]

      C[IND_x_ob,Maximizers]=(C_to_change-0.03*Differences)#*0.9998
      D_Tr[IND_y_ob,Maximizers]=(D_to_change-0.03*Differences)#*0.9998


   # Maximizers = np.argmax(C[IND_x_train]+D_Tr[IND_y_train],-1)
   # B_sparse_ob_est = C[IND_x_train,Maximizers]+D_Tr[IND_y_train,Maximizers]
   # B_sparse_ob=B_sparse_ob_or[range(N)]
   # Differences = B_sparse_ob_est -B_sparse_ob
   # Err[cnt]=np.linalg.norm(Differences)
    #C=C+0.00031
    #D_Tr=D_Tr+0.00031

    Maximizers = np.argmax(C[IND_x_tst]+D_Tr[IND_y_tst],-1)
    B_sparse_ob_est = C[IND_x_tst,Maximizers]+D_Tr[IND_y_tst,Maximizers]
    Differences_tst = B_sparse_ob_est -B_sparse_ob_or_tst

    Maximizers = np.argmax(C[IND_x_val]+D_Tr[IND_y_val],-1)
    B_sparse_ob_est = C[IND_x_val,Maximizers]+D_Tr[IND_y_val,Maximizers]
    Differences_val = B_sparse_ob_est -B_sparse_ob_or_val
    Err_val[cnt]=np.linalg.norm(Differences_val)
    Err_tst[cnt]=np.linalg.norm(Differences_tst)
    if Val_err>Err_val[cnt]:
      Val_err=Err_val[cnt]
      C_op = np.copy(C)
      D_op = np.copy(D_Tr)

    cnt+=1
    D_Tr_32=D_Tr.astype('float32')
    U, s, Vt = np.linalg.svd(D_Tr_32, full_matrices=False)
    Diag_sk_sqrt=np.diag(s[:m]**0.5)
    Uk = U[:, :m]@Diag_sk_sqrt
    Vk = Diag_sk_sqrt@Vt[:m, :]
    D_Tr = (Uk@Vk).astype('float16') #make comment for simple tropical uncomment for mixed


  print(cnt,time.time()-T, 'Val Err' ,np.linalg.norm(Differences_val), 'Tst Err', np.linalg.norm(Differences_tst))

1 21.251084804534912 Val Err 573.9684766711018 Tst Err 574.6910035206612
2 41.91699457168579 Val Err 557.5801441761777 Tst Err 558.3305505706987
3 62.52728533744812 Val Err 546.6558583296346 Tst Err 547.2568817591966
4 82.99002885818481 Val Err 539.1950876025688 Tst Err 539.7951249814777
5 103.80673909187317 Val Err 534.0257539417871 Tst Err 534.4136927975226
6 124.33058595657349 Val Err 529.4375227613598 Tst Err 529.9219601723157
7 144.45441150665283 Val Err 525.6642061829938 Tst Err 525.9509089046896
8 165.34496450424194 Val Err 522.8732970750216 Tst Err 523.1046195963403
9 185.96967959403992 Val Err 520.4551198871284 Tst Err 520.4854395865812
10 205.82357001304626 Val Err 518.5379477135347 Tst Err 518.5346105122183
11 226.77720642089844 Val Err 516.8005492506834 Tst Err 516.872888119563
12 247.44470191001892 Val Err 515.515199600597 Tst Err 515.5188892543916
13 267.4725649356842 Val Err 514.3403591998779 Tst Err 514.2836023697822
14 288.58005690574646 Val Err 513.3526161698969 Tst E

KeyboardInterrupt: ignored

In [26]:
C=np.copy(C_op)
D_Tr = np.copy(D_op)
Maximizers = np.argmax(C[IND_x_tst]+D_Tr[IND_y_tst],-1)
B_sparse_ob_est = C[IND_x_tst,Maximizers]+D_Tr[IND_y_tst,Maximizers]
Differences_tst = B_sparse_ob_est -B_sparse_ob_or_tst
print('RMS',np.linalg.norm(Differences_tst/((N_-N)**0.5)))

RMS 0.3279551553693859


In [27]:
Maximizers = np.argmax(C[IND_x_tst]+D_Tr[IND_y_tst],-1)
B_sparse_ob_est_tst = C[IND_x_tst,Maximizers]+D_Tr[IND_y_tst,Maximizers]
B_tst = B[IND_x_tst,IND_y_tst]
np.linalg.norm(B_sparse_ob_est_tst-B_tst)

Precision_ = 0
Precision_rand=0
for i in range(n):
  AAA=(i==IND_x_tst)
  x_ind = IND_x_tst[AAA]
  y_ind = IND_y_tst[AAA]
  ACTUAL_score = B[x_ind,y_ind]
  Estimated=B_sparse_ob_est_tst[AAA]
  Precision_+= (ACTUAL_score[np.argsort(Estimated)[:10]]==-1).sum()
  Precision_rand+= (ACTUAL_score[np.random.randint(0,ACTUAL_score.shape[0],10)]==-1).sum()

print('Precision@10=',Precision_/(10*n))
print('Precision_@10_RANDOMchoice=',Precision_rand/(10*n))



Precision@10= 0.4208774834437086
Precision_@10_RANDOMchoice= 0.0422682119205298


In [28]:
HR_mat=np.zeros(10)
for epan in range(10):
  count=0
  HIT=0
  for i in range(1,n):
    AAA=(i==IND_x_tst)
    x_ind = IND_x_tst[AAA]
    y_ind = IND_y_tst[AAA]
    ACTUAL_score = B[x_ind,y_ind]
    Estimated=B_sparse_ob_est_tst[AAA]

    AA=np.array(range(ACTUAL_score.shape[0]))
    Positive_all_ind=np.array(ACTUAL_score==-1)
    Positive_all=AA[Positive_all_ind]
    if Positive_all.shape[0]>0:
      Positive_sample = Positive_all[np.random.randint(0,Positive_all.shape[0])]
      Negative_all_ind=np.array(ACTUAL_score==1)
      Negative_all=AA[Negative_all_ind]
      Negative_sample_Ind_of_ind=np.random.choice(Negative_all.shape[0], 100,replace=False)
      Negative_samples = Negative_all[Negative_sample_Ind_of_ind]
      Samples = np.concatenate((np.array([Positive_sample]),Negative_samples), axis=0)
      count+=1
      HIT+=(Estimated[Samples]<Estimated[Samples[0]]).sum() <=10
  HR_mat[epan]=HIT/count

print(HR_mat.mean())

0.7430156328794755
